In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [2]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext
sqlContext=SQLContext(sc)

## 10 categorías que tienen la menor cantidad de contenido anónimo publicado.

Cargo los CSV de contents y de categoryLinks como DF y luego los paso a RDD

In [3]:
df_contents=sqlContext.read.csv('contents.csv', header=True)
df_category=sqlContext.read.csv('categoryLinks.csv', header=True)

In [5]:
rdd_contents=df_contents.rdd.cache()
rdd_category=df_category.rdd.cache()

Hago un map del RDD de category dejando la estructura id como clave y nombre de la categoria como valor

In [7]:
rdd_category_map = rdd_category.map(lambda x: (x[0],x[1]))

Hago un map en el RDD de contents para que me quede la estructura id como clave y las IP como valor ya que me piden todo el contenido anonimo publicado y esto lo hacen usuarios no registrados, por lo tanto que aparece su IP.
Una vez hecho esto, hago un filter por el valor(IP) que no son None asi descarto los ID que pertenecen a contenido publicado por usuarios registrados

In [11]:
rdd_contents_map = rdd_contents.map(lambda x: (x[1],x[8])).filter(lambda x: x[1] != None)

Realizo el Join por ID de las categorias con los contenidos, quedandome la forma id como clave y titulo de la categoria y las ip como valores. Luego realizo un nuevo map moviendo el titulo como clave y un 1 en el valor para poder hacer el ReduceBYKey para que me sumen las repeticiones de cada categoria.

In [23]:
top10_categories = rdd_category_map.join(rdd_contents_map).map(lambda x: (x[1][0],1)).reduceByKey(lambda x,y: x+y)

Realizo el takeOrdered 10 para que me traiga los primeros 10 registros que van a ser los de menor cantidad de publicaciones anonimas

In [24]:
top10_categories.takeOrdered(10, lambda x:x[1])

[('PÃ\xado_X', 1),
 ('Objetos_astronÃ³micos_descubiertos_por_Eleanor_F._Helin', 1),
 ('Localidades_de_Guam', 1),
 ('Teseo', 1),
 ('Reus', 1),
 ('Troya', 1),
 ('TurquÃ\xada_en_1996', 1),
 ('Tratados_de_HungrÃ\xada_del_siglo_XX', 1),
 ('Tratados_de_Polonia_del_siglo_XX', 1),
 ('Tratados_de_Suecia_del_siglo_XX', 1)]

In [11]:
df_contents=sqlContext.read.parquet('contents_p16.parquet')
df_category=sqlContext.read.parquet('categoryLinks.parquet')

In [12]:
rdd_contents=df_contents.rdd.cache()
rdd_category=df_category.rdd.cache()

In [13]:
rdd_category_map = rdd_category.map(lambda x: (x[0],x[1]))
rdd_contents_map = rdd_contents.map(lambda x: (x[1],x[8])).filter(lambda x: x[1] != None)
top10_categories = rdd_category_map.join(rdd_contents_map).map(lambda x: (x[1][0],1)).reduceByKey(lambda x,y: x+y)


In [16]:
top10_categories.takeOrdered(10, lambda x:x[1])

[('PolÃ\x83Â\xadtica_de_la_RestauraciÃ\x83Â³n_borbÃ\x83Â³nica_en_EspaÃ\x83Â±a',
  1),
 ('Socialistas_de_Grecia', 1),
 ('Infantes_de_EspaÃ\x83Â±a_del_siglo_XVIII', 1),
 ('Personas_de_la_guerra_de_sucesiÃ\x83Â³n_austrÃ\x83Â\xadaca', 1),
 ('EpÃ\x83Â³nimos_de_JosÃ\x83Â©_Miguel_Carrera', 1),
 ('Profesores_de_HungrÃ\x83Â\xada', 1),
 ('QuÃ\x83Â\xadmicos_de_HungrÃ\x83Â\xada', 1),
 ('Entrenadores_de_la_UniÃ\x83Â³_Esportiva_Lleida', 1),
 ('Prueba_de_Turing', 1),
 ('AviaciÃ\x83Â³n_de_Corea_del_Sur', 1)]

In [14]:
rdd_category_map.take(2)

[(5, 'Wikipedia:Mantenimiento'),
 (5, 'Wikipedia:PÃ\x83Â¡ginas_con_enlaces_mÃ\x83Â¡gicos_de_ISBN')]

In [15]:
rdd_contents_map.take(4)

[(1133346, '2806:264:3401:1891:8145:B8B8:26C8:F780'),
 (8920516, '2800:E2:1900:951:18A8:5458:910C:FA9A'),
 (1789727, '200.91.9.239'),
 (8390815, '2806:106E:1E:17A7:EDE:1AC1:6547:8B0B')]

In [10]:
rdd_category_map.join(rdd_contents_map).take(4)

[]